<h2> COMP4528 Lab6 </h2>

<p><b>(Acknowledgement: Lab material courtesy of Professor. Du Huynh of UWA).</b></p>

<p>In this lab, we will be working on the camera calibration task.</p>

<p>Four images, stereo2012a.jpg, stereo2012b.jpg, stereo2012c.jpg, and stereo2012d.jpg, are given for this lab. These images are different views of a calibration target and some objects. For example, the diagram below is stereo2012a.jpg with some text superimposed onto it:</p>

<p align="center">
  <img src="notebook_assets/1.png">
</p>

<p><b>On the calibration target there are 3 mutually orthogonal faces. The points marked on each face form a regular grid. They are all 7cm apart.</b></p>

<p>From the 4 supplied images (stereo2012a.jpg, stereo2012b.jpg, stereo2012c.jpg, and stereo2012d.jpg), <font color="red">choose any image to work on</font>. Use the suggested right-hand coordinate system shown in the diagram above and choose a sufficient number of calibration points on the calibration target.</p>

<p>Store the XYZ coordinates in a file so that you can load the data into Python and use them again and again. Note that each image can be calibrated independently, so you can choose different calibration points to calibrate each image. Neither do the numbers of calibration points need to be the same for your chosen images. <font color="red">Read through the choose_points.py script in the lab materials, understand the code, run the script to generate the XYZ and uv coordinates and store them in the data folder.</font></p>

<p>After the above operation, the variable uv should be a 12 × 2 matrix, each row of which should contain the coordinates of one image point.</p>

<p><font color="red"> Note: You need to ensure that, for each image, the numbers of 3D and 2D calibration points are the same. For example, if your uv variable is a 12 × 2 matrix, then your XYZ variable should be a 12 × 3 matrix. Also, the points should appear in the same order in these two matrices. </font></p>

<p>Use the DLT algorithm to solve the unknown camera calibration matrix of size 3x4. (Refer to lecture slides and textbook: Multiple View Geometry in Computer Vision Section 7 (page 178)).</p>


<h3> Hints: </h3>

<ol>
  <li>In writing your code you may need to "reshape" a 2D vector into a 1D vector. You can use the function np.reshape to reshape a matrix to arbitrary dimensions.</li>
  <li>You can save your calibration matrices using NumPy save command. Read the following links for more details.
  <ul>
    <li><a href="url">https://numpy.org/doc/stable/reference/generated/numpy.save.html</a></li>
    <li><a href="url">https://numpy.org/doc/stable/reference/generated/numpy.load.html</a></li>
  </ul>
  </li>
</ol>

### Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt


### Utility functions (you don't have to understand implementation details)

In [2]:
'''
VGG_KR_FROM_P Extract K, R from camera matrix.

[K,R,C] = VGG_KR_FROM_P(P [,noscale]) finds K, R, C such that P = K*R*[eye(3) -C].
It is det(R)==1.
K is scaled so that K(3,3)==1 and K(1,1)>0. Optional parameter noscale prevents this.

Works also generally for any P of size N-by-(N+1).
Works also for P of size N-by-N, then C is not computed.

Original Author: Andrew Fitzgibbon <awf@robots.ox.ac.uk> and awf
Date: 15 May 98

Modified by Shu.
Date: 8 May 20
'''


def vgg_rq(S):
    S = S.T
    [Q, U] = np.linalg.qr(S[::-1, ::-1], mode='complete')

    Q = Q.T
    Q = Q[::-1, ::-1]
    U = U.T
    U = U[::-1, ::-1]
    if np.linalg.det(Q) < 0:
        U[:, 0] = -U[:, 0]
        Q[0, :] = -Q[0, :]
    return U, Q


def vgg_KR_from_P(P, noscale=True):
    N = P.shape[0]
    H = P[:, 0:N]
    print(N, '|', H)
    [K, R] = vgg_rq(H)
    if noscale:
        K = K / K[N-1, N-1]
        if K[0, 0] < 0:
            D = np.diag([-1, -1, np.ones([1, N-2])])
            K = K @ D
            R = D @ R

            test = K*R
            assert (test/test[0, 0] - H/H[0, 0]).all() <= 1e-07

    C = np.linalg.inv(-P[:, 0:N]) @ P[:, -1]
    return K, R, C


### Load the data from saved npy files

In [3]:
xyz = np.load("data/stereo2012a_xyz.npy")
uv = np.load("data/stereo2012a_uv.npy")
img = plt.imread("images/stereo2012a.jpg")


### XYZ normalisation matrix

The normalisation matrix for XYZ coordinates $\mathbf{S}_{\text{norm}}$ is defined as $$\mathbf{S}_{\text{norm}} = \begin{bmatrix}\mathbf{V}\text{diag}(\lambda_1^{-1}, \lambda_2^{-1}, \lambda_3^{-1}) \mathbf{V}^{-1} & -\mathbf{V}\text{diag}(\lambda_1^{-1}, \lambda_2^{-1}, \lambda_3^{-1}) \mathbf{V}^{-1}\mu_\mathbf{x}\\ 0 & 1 \end{bmatrix}$$

where $$\mathbf{V}\text{diag}(\lambda_1, \lambda_2, \lambda_3) \mathbf{V}^{-1} = \text{eig}(\sum_i(\mathbf{X}_{i,\text{nonhom}} - \mu_\mathbf{x})^\top(\mathbf{X}_{i,\text{nonhom}} - \mu_\mathbf{x}))$$
$$\mathbf{X}_{\text{nonhom}} = \begin{bmatrix}x_1 & y_1 & z_1\\ x_2 & y_2 & z_2\\ \vdots & \vdots & \vdots\\ x_N & y_N & z_N\\  \end{bmatrix}$$
$$\mu_\mathbf{x} = \begin{bmatrix}\frac{1}{N}\sum_ix_i & \frac{1}{N}\sum_iy_i & \frac{1}{N}\sum_iz_i\end{bmatrix}$$

In [4]:
# Generate the normalisation matrix S for XYZ coordinate
def xyz_normalisation_mat(xyz):
    # calculate the mean across all rows
    mu_x = xyz.mean(axis=0)

    # matrix storing the sum of covariance
    diff = xyz - mu_x
    cov_mat = diff.T @ diff

    # eigen-decomposition the covariance matrix
    w, v = np.linalg.eig(cov_mat)

    # S for storing the normalisation matrix
    s = np.zeros((4, 4))

    # diagnol matrix should be the reciprocal of eigenvalues
    diagnol_mat = np.diag(np.reciprocal(w))

    # constructing S matrix
    v_inv = np.linalg.inv(v)
    s[0:3, 0:3] = v @ diagnol_mat @ v_inv
    s[0:3, 3:4] = -v @ diagnol_mat @ v_inv @ mu_x.reshape((-1, 1))
    s[3, 3] = 1
    return s


### uv normalisation matrix

The normalisation matrix for uv coordinates $\mathbf{T}_{\text{norm}}$ is defined as $$\mathbf{T}_{\text{norm}} = \begin{bmatrix}w + h & 0 & \frac{w}{2}\\[1ex]  0 & w + h & \frac{h}{2}\\[1ex] 0 & 0 & 1\\\end{bmatrix}^{-1}$$

In [5]:
def uv_normalisation_mat(h, w):
    return np.linalg.inv(
        np.array([
            [w + h, 0, w / 2],
            [0, w + h, h / 2],
            [0, 0, 1]
        ])
    )


### Calibrate

Function to perform camera calibration

**Usage:**   

```calibrate(img, xyz, uv)```

```return p_mat```

**Where:**  
 - img: the image of the calibration target.
 - xyz: a N x 3 array of  XYZ coordinates of the calibration target points. 
 - uv: a N x 2 array of the image coordinates of the calibration target points.
 - p_mat: the 3 x 4 camera calibration matrix.


The variable N should be an integer greater than or equal to 6.

**The root mean squared error (RMSE) between the positions of the uv coordinates and the projected XYZ coordinates should be reported.**


In [6]:
def calibrate(img, xyz, uv):
    # pre-condition checking
    assert xyz.shape[0] >= 6 and xyz.shape[1] == 3
    assert uv.shape[0] >= 6 and uv.shape[1] == 2
    assert xyz.shape[0] == uv.shape[0]

    h, w, _ = img.shape

    # coordinate normalisation matrix
    t_mat = uv_normalisation_mat(h, w)
    s_mat = xyz_normalisation_mat(xyz)

    # transform uv to homogeneous and normalise
    uv_normalised = np.hstack((uv, np.ones((uv.shape[0], 1))))
    uv_normalised = (t_mat @ uv_normalised.T).T

    # transform XYZ to homogeneous and normalise
    xyz_normalised = np.hstack((xyz, np.ones((xyz.shape[0], 1))))
    xyz_normalised = (s_mat @ xyz_normalised.T).T

    # A matrix with size (N * 2) x 12
    a_mat = np.zeros((xyz_normalised.shape[0] * 2, 12))

    # iterate through corresponding points
    for i, x_i in enumerate(xyz_normalised):
        # each set of corresponding points can generate two equations
        basic_block = np.array([x_i[0], x_i[1], x_i[2], 1])
        block1 = basic_block * uv_normalised[i][2]
        block2 = basic_block * uv_normalised[i][1]
        block3 = -basic_block * uv_normalised[i][0]
        zero_block = np.zeros(4)
        part = np.vstack((
            np.hstack((zero_block, -block1, block2)),
            np.hstack((block1, zero_block, block3)),
        ))
        a_mat[2 * i: 2 * (i + 1)] = part

    # SVD to matrix A. p should be the last column of matrix V
    v = np.linalg.svd(a_mat)[2]
    p = v[-1, :]
    # normalize vector p and rearrange it into matrix P
    p_normalized = p / np.linalg.norm(p)
    p_mat = np.reshape(p_normalized, (3, 4))

    # denormalize to get the calibration matrix
    p_mat = np.linalg.inv(t_mat) @ p_mat @ s_mat

    # calculate the projection of world coordinates
    projection_homo = (
        p_mat @ (np.hstack((xyz, np.ones((xyz.shape[0], 1))))).T).T

    # transform from homogeneous coordinate to normal coordinate
    projection = projection_homo[:, :2] / projection_homo[:, 2:3]

    # RMSE calculation
    rmse = np.sqrt(((projection - uv) ** 2).mean())
    print(
        f"The RMSE for projection of XYZ coordinate to uv coordinate is {rmse} (in pixel unit)")

    return p_mat


### Calibrate the camera and decompose the calibration matrix with util functions

1. Obtain the camera calibration matrix with the `calibrate` function defined above
2. Decompose the $\mathbf{P}$ matrix into $\mathbf{K}$, $\mathbf{R}$, $\mathbf{C}$ using the util function defined above, such that $\mathbf{P} = \mathbf{K}[\mathbf{R}|-\mathbf{R}\mathbf{C}]$.

In [ ]:
p_mat = calibrate(img, xyz, uv)
k_mat, r_mat, C = vgg_KR_from_P(p_mat)

print(f"P: \n {p_mat}")
print(f"K: \n {k_mat}")
print(f"R: \n {r_mat}")
print(f"C: \n {C}")


The RMSE for projection of XYZ coordinate to uv coordinate is 0.14469181590276978 (in pixel unit)
3 | [[ 1.96374876e+00 -8.80582449e-01 -2.90352580e+00]
 [-1.63198872e-02 -3.39254322e+00  6.29457748e-01]
 [-2.14123071e-03 -1.39798179e-03 -2.73017450e-03]]
P: 
 [[ 1.96374876e+00 -8.80582449e-01 -2.90352580e+00  1.53272188e+02]
 [-1.63198872e-02 -3.39254322e+00  6.29457748e-01  1.58427918e+02]
 [-2.14123071e-03 -1.39798179e-03 -2.73017450e-03  4.76828132e-01]]
K: 
 [[898.97608883   3.57547745 353.98446902]
 [  0.         896.12720343 218.61733953]
 [  0.           0.           1.        ]]
R: 
 [[ 0.80881572 -0.11103809 -0.57748391]
 [ 0.1347754  -0.92087177  0.3658289 ]
 [-0.57240957 -0.37371879 -0.72985036]]
t: 
 [74.57029328 62.0032357  84.41824339]


### Discussion questions
Use the results from above to answer the following questions
1. What is the focal length (in the unit of a pixel) of the camera?
2. What is the camera translation $\mathbf{t}$? That is, \mathbf{t} such that $\mathbf{P} = \mathbf{K}[\mathbf{R}|\mathbf{t}]$.

<b><font color="red">Sample answer:</font></b>
1. $$\mathbf{K} = \begin{bmatrix}
            \alpha & \gamma & u_0\\
            0 & \beta & v_0\\
            0 & 0 & 1\\
        \end{bmatrix}$$
    where
    $$\begin{align*}
        \alpha &= f_x = 898.98\\
        \beta &= \frac{f_y}{\sin{(\theta)}}\\
        \gamma &= -f_x\cot{(\theta)}
    \end{align*}$$
    Calculate the skew factor $\theta$
    $$\theta = \arctan{(\frac{-f_x}{\gamma})} = \arctan{(\frac{898.98}{3.5755})} = \arctan{(251.42)} = 1.567 + k\pi \quad (k \in \mathcal{Z})$$
    Since $\theta \in [0, \pi]$, the possible value for skew could be $\theta = 1.567$.

    Then, we can calculate the focal length on the $y$-axis
    $$f_y = \beta\sin{(\theta)} = 896.13 \times \sin{(1.567)} = 896.12$$

    So, we can conclude that the focal length for $x$-axis is $fx = 898.98$ pixel units and the focal length for $y$-axis is $f_y = 896.12$ pixel units.
2. The camera centre coordinate is denoted by $\mathbf{C}$
   $$\begin{align*}
        \mathbf{t} &= -\mathbf{R}\mathbf{C}\\
        &= - \begin{bmatrix}
            0.8088 & -0.1110 & -0.5775\\
            0.1348 &  -0.9209 & 0.3658\\
            -0.5724 & -0.3737 & -0.7299\\
        \end{bmatrix}
        \begin{bmatrix}
            74.5703\\
            62.0032\\
            84.4182\\
        \end{bmatrix}\\
        &= \begin{bmatrix}
            -4.6786\\
            16.1665\\
            127.4715\\
        \end{bmatrix}
    \end{align*}$$
    So, the camera translation is $(-4.6786, 16.1665, 127.4715)$.